In [1]:
import pandas as pd
import numpy as np
import os
from tabula import read_pdf
import re
from dateutil import parser
from datetime import datetime
from datetime import date

# pdf examples 090520 080920 040720 120420 170620 210320 240820

In [2]:
# READ CSV

def get_table(file_number):

    df = pd.read_csv(directory + '/tabula-DL' + file_number + '.csv')
    
    return df

In [3]:
# GET INDEX OF 'ACEH'

def getIndexes(dfObj, value):
    listOfPos = list()
    result = dfObj.isin([value])
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    return listOfPos

In [4]:
def process_table(df):

    pos_aceh = getIndexes(df,'ACEH')[0][0]

    # DROP SOME UNUSED ROWS AND COLUMNS

    drop_columns1 = [df.columns[0], df.columns[2]]
    drop_rows = list(range(0, pos_aceh))
    df = df.drop(drop_rows)
    df = df.drop(labels=drop_columns1, axis=1)

    # RENAME COLUMNS 'PROVINSI' AND COLUMS THAT CONTAIN THE NUMBERS

    for i in range(len(df.columns)):
        if i == 0 : df.rename(columns={df.columns[i]: 'PROVINSI'}, inplace=True)
        else :
                df.rename(columns={df.columns[i]: 'TEMP' + str(i)}, inplace=True)

    # SPLIT THE NUMBERS IN THE CONTAINING COLUMNS AND PLACE THE NUMBERS ON SEPARATE COLUMNS

    for i in np.arange(1, len(df.columns)):
        s = df['TEMP' + str(i)].str.split(' ', expand = True)
        y = 0
        for z in np.arange(len(s.columns)):
            df[str(i) + '-' + str(z)] = s[y]
            y = y + 1

    # DROP COLUMNS CONTAINING ONLY RESULTING FROM THE SPLITTING PROCESS

    df.replace("", np.nan, inplace=True)
    df.dropna(how='all', axis=1, inplace=True)

    # DROP TEMP COLUMNS

    columns_list = list(df.columns)
    columns_list
    temp_index = [i for i, word in enumerate(columns_list) if word.startswith('TEMP')]
    temp_drops = []
    
    for i in temp_index:
        temp_drops.append(df.columns[i])

    df = df.drop(temp_drops, axis=1)

    # RENAME 'TOTAL' ROW AND ADD 'TGL' COLUMN

    df.iloc[-1][0] = 'TOTAL'
    df["TGL"] = file_number[:2] + "-" + file_number[2:4] + "-" + file_number[4:]

    [parser.parse(i) for i in df['TGL']]
    df['TGL'] = [datetime.strptime(i, '%d-%m-%y') for i in df['TGL']]
    df['TGL'] = [i.date() for i in df['TGL']]

    # RENAME NUMBER COLUMNS NAMES

    number_column_names = ['PSTF H-1', 'PSTF', 'PSTF KUM', 'SMBH H-1', 'SMBH', 'SMBH KUM',
                        'MNGL H-1', 'MNGL', 'MNGL KUM']

    for i in np.arange(1, len(df.columns)-1):
        df = df.rename(columns={df.columns[i]:number_column_names[i-1]})

    # CLEAN ROWS AT THE END

    pos_gorontalo = getIndexes(df,'GORONTALO')[0][0]
    pos_total = getIndexes(df, 'TOTAL')[0][0]
    pos_total

    drop_rows_end = np.arange(pos_gorontalo + 1, pos_total)
    df = df.drop(drop_rows_end)

    # REMOVE MULTIPLE WHITESPACES ON 'PROVINSI'
    
    df = df.replace(to_replace=r'\s\s+', value=' ', regex=True)
    
    # RESET INDEX

    df = df.reset_index(drop=True)

    # CONVERT ALL COLUMN TYPES

    convert_columns = {'PSTF H-1': int, 'PSTF': int, 'PSTF KUM': int, 'SMBH H-1': int, 'SMBH': int,
                       'SMBH KUM': int, 'MNGL H-1': int, 'MNGL': int, 'MNGL KUM': int} 
    df = df.astype(convert_columns)
    
    return df

In [5]:
def clean_table(df):
    
    # DROP MORE UNUSED ROWS AND COLUMNS

    drop_columns2 = [df.columns[1], df.columns[4], df.columns[7]]
    df_clean = df.drop(labels=drop_columns2, axis=1)
    df_clean = df_clean.drop(getIndexes(df,'TOTAL')[0][0])
    
    return df_clean

In [6]:
def clean_to_excel(df_clean):
    
#  SAVE CLEAN DATA TABLE TO EXCEL FILE

    df_clean.to_excel(directory + '/CL' + file_number + '.xlsx')
    print('processing CL' + file_number + ' done!')


In [7]:
def process_all(files):
    df = get_table(file_number)
    df = process_table(df)
    df_clean = clean_table(df)
    clean_to_excel(df_clean)
    return df_clean

In [8]:
error_files = []
error_msgs = []

# SET THE FOLDER NAME

month_name = 'Mei'
directory = 'TabulaCSV/' + month_name

# PREPARE ARRAYS WHERE THE DATA IS POPULATED

clean_columns = ['PROVINSI', 'PSTF', 'PSTF KUM', 'SMBH', 'SMBH KUM', 'MNGL', 'MNGL KUM', 'TGL']

agg_df_clean = pd.DataFrame(columns=clean_columns)

# IMPLEMENT FUNCTION WITH ITERATION OF FILES IN FOLDER

for file in os.listdir(directory):
    file_number = file[9:15]
    print('Processing ' + file_number)
    try:
        df_clean = process_all(file_number)
        agg_df_clean = agg_df_clean.append(df_clean, ignore_index=True)
    except Exception as e:
        error_files.append(file_number)
        error_msgs.append(e)
        print(e)
        continue
        
# HANDLE LOG OF ERRORS

error_dict = {'Error Files': error_files, 'Error Messages': error_msgs}
df_error = pd.DataFrame(data=error_dict)
df_error.to_excel(directory + '/ErrorLog_' + month_name + '.xlsx')

# WRITE AGGREGATED DATA TO EXCEL

agg_df_clean.to_excel(directory + '/AggCL_' + month_name + '.xlsx')

Processing 110520
processing CL110520 done!
Processing 270520
processing CL270520 done!
